<h1 style="font-size:2.5em;text-align:center;">Sistemas de Recomendación - Guia 2</h1>

# Implementación de un FWLS

En este notebook veremos los pasos para crear (y evaluar) un modelo de Ensemble de **Feature Weighted Linear Stacking** basado en dos sistemas de recomendación: un filtrado colaborativo y un filtrado por contenido.

# Modelo de Filtrado Colaborativo

Para el primer modelo vamos a utilizar [Surpr!se](http://surpriselib.com/) como lo venimos haciendo. Además, haremos uso de su sistema de `Dataset`s que nos serviará para la división del conjunto de datos en entrenamiento y evaluación.

In [1]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 5.4 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633719 sha256=a477afa82894be7e3c42e87e6cf788252186c818789f6c1ea75cd435fa1b6fa7
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [1]:
!curl -LO http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip -d data/
!rm -f ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 97  955k   97  931k    0     0   705k      0  0:00:01  0:00:01 --:--:--  708k
100  955k  100  955k    0     0   722k      0  0:00:01  0:00:01 --:--:--  725k
'unzip' is not recognized as an internal or external command,
operable program or batch file.
'rm' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import pandas as pd

from surprise import Dataset, Reader, KNNWithMeans
from surprise.model_selection import train_test_split

In [4]:
ratings = pd.read_csv("ml-latest-small/ratings.csv")
reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))
ratings = Dataset.load_from_df(ratings[["userId", "movieId", "rating"]], reader)
ratings_train, ratings_test = train_test_split(ratings, test_size=0.2)

Notar que en este casos el filtrado es basado en items (películas), en lugar de usuarios.

In [5]:
cf_model = KNNWithMeans(k=5, sim_options={"user_based": False, "name": "pearson"}).fit(ratings_train)

Computing the pearson similarity matrix...
Done computing similarity matrix.


# Modelo predictivo basado en contenido

Primero debemos crear nuestro modelo predictivo basado en información de contenido. En este caso, basado en los géneros de las películas. La idea es, a partir de los géneros de las películas, buscar aquellas más similares. Luego, en base a los ratings hechos por un usuario, se pueden generar ratings usando algún algoritmo clásico de KNearestNeigbors y en base a ello devolver lo predicho.

In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

class GenresBasedFilter(object):
    def __init__(self, movies, k=5):
        self.movie_to_idx = {row["movieId"]: idx for idx, row in movies.iterrows()}
        self.idx_to_movie = {idx: movie for movie, idx in self.movie_to_idx.items()}
        self.k = k

        genres = set(g for G in movies['genres'] for g in G)
        for g in genres:
            movies[g] = movies.genres.transform(lambda x: int(g in x))

        self.movie_genres = movies.drop(columns=['movieId', 'title', 'genres'])

    def fit(self, ratings):
        self.movies_cosine_sim_ = cosine_similarity(self.movie_genres, self.movie_genres)

        self.user_ratings_ = {}
        for (user_id, movie_id, rating) in ratings.build_testset():
            if user_id not in self.user_ratings_:
                self.user_ratings_[user_id] = {}
            self.user_ratings_[user_id][movie_id] = rating

        return self

    def predict(self, user, movie):
        if not user in self.user_ratings_ or not movie in self.movie_to_idx:
            global_mean = np.mean([
                rating for movies in self.user_ratings_.values() for rating in movies.values()
            ])
            return global_mean

        movie_idx = self.movie_to_idx[movie]
        sim_scores = list(enumerate(self.movies_cosine_sim_[movie_idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:]

        sims = []

        for movie, score in sim_scores:
            if self.idx_to_movie[movie] in self.user_ratings_[user]:
                sims.append((self.user_ratings_[user][self.idx_to_movie[movie]], score))
                if len(sims) >= self.k:
                    break

        user_mean = np.mean(list(self.user_ratings_[user].values()))

        pred = 0
        sim_sum = 0

        for rating, score in sims:
            pred += score * (rating - user_mean)
            sim_sum += score

        if sim_sum == 0:
            return user_mean

        return user_mean + pred / sim_sum

Para este modelo cargamos el conjunto de datos que tiene información de los géneros de las películas. Necesitamos esta información para calcular la matriz de similitud coseno. Por otra parte, utilizamos el conjunto de entrenamiento de Surpr!se para entrenar el modelo.

In [7]:
movies = pd.read_csv("ml-latest-small/movies.csv")
movies['genres'] = movies['genres'].apply(lambda x: x.split("|"))

In [8]:
cb_model = GenresBasedFilter(movies).fit(ratings_train)

# Feature Weighted Linear Stacking

## Meta atributos de modelos

Con nuestros dos modelos bases, pasamos a crear nuestro modelo FWLS. Para ello, lo primero que tenemos que hacer es transformar los meta atributos de los modelos a partir de la información de usuario y película (esto lleva un tiempo, porque el algoritmo de filtrado por contenido no está optimizado).

In [13]:
from tqdm import tqdm

transformed_ratings_train = []

for u, m, r in tqdm(ratings_train.build_testset()):
    transformed_ratings_train.append({
        "userId": u,
        "movieId": m,
        "cb_rating": cb_model.predict(u, m),
        "cf_rating": cf_model.predict(u, m).est,
        "rating": r
    })

transformed_ratings_train = pd.DataFrame(transformed_ratings_train)

100%|██████████| 80668/80668 [06:31<00:00, 206.10it/s]


## Funciones atributo

El siguiente paso se basa en definir nuestras funciones atributo que se le darán en conjunto a los meta atributos de los modelos para entrenar el algoritmo de regresión logística. En este caso solo definiremos tres muy sencillas.

In [14]:
from collections import defaultdict

user_mean_rating = defaultdict(
    lambda: transformed_ratings_train["rating"].mean(),
    transformed_ratings_train.groupby("userId")["rating"].mean().to_dict()
)
user_num_rating = defaultdict(
    lambda: 0,
    transformed_ratings_train.groupby("userId").size().to_dict()
)

def feature_function_constant():
    return 1

def feature_function_mean(user_id):
    return user_mean_rating[user_id]

def feature_function_over(user_id, min_ratings=3):
    return int(user_num_rating[user_id] >= min_ratings)

## Generación de atributos de entrenamiento

A partir de nuestros meta atributos de lo modelos y nuestras funciones atributo, podemos definir nuestros atributos finales que serán utilizados en el modelo de regresión logística. Para ello tenemos que aplicar las funciones atributo a los meta atributos de los modelos.

In [15]:
for base_model in ["cb", "cf"]:
    transformed_ratings_train["{}_rating_fc".format(base_model)] =\
        transformed_ratings_train.apply(
            lambda row: row["{}_rating".format(base_model)] * feature_function_constant(),
            axis=1
        )
    transformed_ratings_train["{}_rating_fm".format(base_model)] =\
        transformed_ratings_train.apply(
            lambda row: row["{}_rating".format(base_model)] * feature_function_mean(row["userId"]),
            axis=1
        )
    transformed_ratings_train["{}_rating_fo".format(base_model)] =\
        transformed_ratings_train.apply(
            lambda row: row["{}_rating".format(base_model)] * feature_function_over(row["userId"]),
            axis=1
        )

## Entrenamiento del modelo

El último paso es el entrenamiento del modelo de regresión lineal en base a nuestros atributos generados en el paso anterior. Este es el paso más sencillo porque utilizamos directamente el algoritmo de `scikit-learn` para regresión lineal.

In [16]:
from sklearn.linear_model import LinearRegression

fwls_model = LinearRegression()

feature_cols = ["{}_rating_{}".format(fm, ff) for fm in ["cb", "cf"] for ff in ["fc", "fo", "fm"]]

fwls_model.fit(
    transformed_ratings_train[feature_cols],
    transformed_ratings_train["rating"]
)

LinearRegression()

## Evaluación del modelo

Para evaluar los modelos tenemos que realizar el mismo paso de transformación sobre los datos de evaluación que se utilizaron para los datos de entrenamiento. Empezando por la obtención de meta atributos de los modelos.

In [18]:
transformed_ratings_test = []

for u, m, r in tqdm(ratings_test):
    transformed_ratings_test.append({
        "userId": u,
        "movieId": m,
        "cb_rating": cb_model.predict(u, m),
        "cf_rating": cf_model.predict(u, m).est,
        "rating": r
    })

transformed_ratings_test = pd.DataFrame(transformed_ratings_test)

100%|██████████| 20168/20168 [01:38<00:00, 205.71it/s]


Y siguiendo por los atributos a partir de las funciones atributo.

In [19]:
for base_model in ["cb", "cf"]:
    transformed_ratings_test["{}_rating_fc".format(base_model)] =\
        transformed_ratings_test.apply(
            lambda row: row["{}_rating".format(base_model)] * feature_function_constant(),
            axis=1
        )
    transformed_ratings_test["{}_rating_fm".format(base_model)] =\
        transformed_ratings_test.apply(
            lambda row: row["{}_rating".format(base_model)] * feature_function_mean(row["userId"]),
            axis=1
        )
    transformed_ratings_test["{}_rating_fo".format(base_model)] =\
        transformed_ratings_test.apply(
            lambda row: row["{}_rating".format(base_model)] * feature_function_over(row["userId"]),
            axis=1
        )

transformed_ratings_test["fwls_rating"] = fwls_model.predict(transformed_ratings_test[feature_cols])

## Comparación de los modelos

Finalmente, tenemos todo lo necesario para hacer una comparación (e.g. midiento el error) de los distintos modelos sobre el conjunto de evaluación.

In [20]:
from sklearn.metrics import mean_squared_error

for model in ["cf", "cb", "fwls"]:
    rmse = np.sqrt(
        mean_squared_error(
            transformed_ratings_test["rating"],
            transformed_ratings_test["{}_rating".format(model)]
        )
    )
    
    print("RMSE for {} model: {:03f}".format(model, rmse))

RMSE for cf model: 0.957608
RMSE for cb model: 0.967298
RMSE for fwls model: 0.948018
